#### **Indeed Python job scraper**

In [132]:
import requests # http requests
from bs4 import BeautifulSoup # Webscrape
import pandas as pd     # DF

JOB_SEARCH = 'Data scientist'
JOB_LOCATION = 'Buenos Aires'
HEADER = {"User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}
N_PAGES = 10

In [133]:
def get_indeed_jobs(search:str,location:str,n_pages:int)->list:
    
    """
        Get principal elements of each job result including the job company, title, description link, description, job_type and date.
        
        Returns a list of dictionaries containing the above elements for every job result.
    """
    
    BASE_URL = 'https://ar.indeed.com'
    posts=[]
    
    skill = JOB_SEARCH.strip()
    place = JOB_LOCATION.strip()
    no_of_pages = N_PAGES

    for page in range(no_of_pages):

            # Connecting to Arg Indeed
            url = BASE_URL + '/jobs?q=' + skill + \
                '&l=' + place + '&sort=date' +'&start='+ str(page * 10)

            # Get request to indeed with headers above 
            response = requests.get(url, headers=headers)
            html = response.text

            # Scrapping the Web 
            soup = BeautifulSoup(html, 'lxml')

            # Outer Most Entry Point of HTML:
            outer_most_point=soup.find('div',attrs={'id': 'mosaic-provider-jobcards'})

            # "UL" lists where the data is stored:
            for i in outer_most_point.find('ul'):

            # get Job Title:

                job_content=i.find('td',{'class':'resultContent'})
                job_title = None
            
                if job_content != None:
                    job_title=job_content.find('a').text
                    job_location = job_content.find('div',{'class':'companyLocation'}).text
                if job_title==None:
                    job_title='None'
                if job_location==None:
                    job_location='None'

            # get Company Name:

                if i.find('span',{'class':'companyName'}) != None:
                    company=i.find('span',{'class':'companyName'}).text
                else:
                    company='None'

            # get Links: these Href links will take us to full job description

                if i.find('a') != None:
                    desc_link= i.find('a',{'class':'jcs-JobTitle'})['href']
                    if desc_link!=None:
                        url_href= BASE_URL + desc_link
                        # Get job description
                        response = requests.get(url_href, headers=headers)
                        html_ = response.text
                        soup_ = BeautifulSoup(html_, 'lxml')
                        job_descr = soup_.find('div',{'class':'jobsearch-jobDescriptionText'}).text.strip()
                        if job_descr==None:
                            job_descr='None'
                else:
                    desc_link='None'

            # get job_type if available:

                if i.find('div',{'class':'attribute_snippet'}) != None:
                    job_type=i.find('div',{'class':'attribute_snippet'}).text

      
                else:
                    job_type='None'

            # get Job Post Date:

                if i.find('span', attrs={'class': 'date'}) != None:
                    post_date = i.find('span', attrs={'class': 'date'}).text
                else:
                    post_date='None'

            # Put everything together in a list of dicts

                posts.append({'Company':company,'Location':job_location,'Title':job_title,'Description_link':BASE_URL + desc_link,'Description':job_descr,'Job_type':job_type, 'Posted_date':post_date})

            return posts

In [134]:
posts = get_indeed_jobs(JOB_SEARCH,JOB_LOCATION,N_PAGES)

In [135]:
print('Example of a job link description')
print('\n')

df = pd.DataFrame(posts)

print(df['Description_link'][0]) 

df

Example of a job link description


https://ar.indeed.com/rc/clk?jk=1817d0354ad28bd4&fccid=a3f737e511d9fc8c&vjs=3


,Company,Location,Title,Description_link,Description,Job_type,Posted_date
0,Visa,"Buenos Aires, Buenos Aires",Data Scientist - Manager,https://ar.indeed.com/rc/clk?jk=1817d0354ad28b...,Company Description\n As the world's leader i...,Tiempo completo,PostedHoy
1,Accenture,"Buenos Aires, Buenos Aires",P&A - Data Scientist - Associate Manager,https://ar.indeed.com/rc/clk?jk=5a3eef7bb8edbc...,OBJECTIVE\n\n Be part of a multi-disciplinary ...,Tiempo completo,PostedHoy
2,Accenture,"Buenos Aires, Buenos Aires",BDS - ML11 Analyst,https://ar.indeed.com/rc/clk?jk=75b7bd5bed004e...,Quien se sume al equipo como Analista de Data ...,Tiempo completo,PostedHoy
3,Accenture,"Buenos Aires, Buenos Aires",Business Associate Manager for Data Analytics ...,https://ar.indeed.com/rc/clk?jk=3d92331d1607ad...,OBJECTIVE\n\n Be part of a multi-disciplinary ...,Tiempo completo,PostedHoy
4,Jampp,"Buenos Aires, Buenos Aires",Senior Data Scientist,https://ar.indeed.com/rc/clk?jk=023f6554659d08...,Jampp is looking for a Senior Data Scientists ...,None,PostedPublicado hace 2 días
5,None,"Buenos Aires, Buenos Aires",None,https://ar.indeed.comNone,Jampp is looking for a Senior Data Scientists ...,None,None
6,Jampp,"Buenos Aires, Buenos Aires",Senior Product Manager,https://ar.indeed.com/rc/clk?jk=57769dda9ebfc2...,ABOUT JAMPP\n Jampp is a programmatic advertis...,None,PostedPublicado hace 2 días
7,Selin,"Buenos Aires, Buenos Aires",Data Scientist Sr . Compañía editorial,https://ar.indeed.com/rc/clk?jk=633d633e4d4b10...,"Para una importante compañía editorial, nos en...",None,PostedPublicado hace 5 días
8,Worley,"Buenos Aires, Buenos Aires",Senior Piping Stress Engineer - Buenos Aires,https://ar.indeed.com/rc/clk?jk=ae7b056b0a0d31...,"48,000 people. 49 countries. Over 120 office l...",Tiempo completo,PostedPublicado hace 6 días
9,Worley,"Buenos Aires, Buenos Aires",Senior Procurement Contract Specialist,https://ar.indeed.com/rc/clk?jk=c8e71279653739...,"48,000 people. 49 countries. Over 120 office l...",Tiempo completo +1,PostedPublicado hace 6 días
